In [1]:
#! pip install pyarrow matplotlib sentencepiece pandas
import torch
import os
import sys
import numpy as np
import math
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.nn import TransformerDecoder, TransformerDecoderLayer
import sentencepiece as spm
import matplotlib.pyplot as plt
import multiprocessing
import time
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

if torch.backends.mps.is_available():  # Check for Apple Silicon GPU availability (requires PyTorch 1.12 or later)
    device = torch.device("mps")
elif torch.cuda.is_available():  # Check for NVIDIA GPU availability
    device = torch.device("cuda")
    torch.cuda.empty_cache()
else:
    device = torch.device("cpu")  # Fall back to CPU

print(f"Using device: {device}")

Using device: mps


Dataset Import

In [20]:
# Read data from both Parquet files
train_0 = pd.read_parquet('../dataset/0000.parquet')
train_1 = pd.read_parquet('../dataset/0001.parquet')
train_2 = pd.read_parquet('../dataset/0002.parquet')
train_3 = pd.read_parquet('../dataset/0003.parquet')
train_4 = pd.read_parquet('../dataset/0004.parquet')
train_5 = pd.read_parquet('../dataset/0005.parquet')
train_6 = pd.read_parquet('../dataset/0006.parquet')
train_7 = pd.read_parquet('../dataset/0007.parquet')
train_8 = pd.read_parquet('../dataset/0008.parquet')

train = pd.concat([train_0, train_1, train_2, train_3, train_4, train_5, train_6, train_7, train_8,], ignore_index=True)
train = train.reset_index(drop=True)

# Concatenate elements within each row
train['caption'] = train['caption'].apply('\n'.join)
# Concatenate all rows into one long text string
long_text = train['caption'].str.cat(sep='\n')
with open('../dataset/corpus.txt', 'w') as file:
    file.write(long_text)

# Train SentencePiece model directly from Python list of sentences
spm.SentencePieceTrainer.train(input='../dataset/corpus.txt', model_prefix='../dataset/spm_10000_vocab_model', vocab_size=10000, model_type='unigram')

print(train.shape)
print(train['caption'])
train = train.iloc[:100]

In [18]:
sp = spm.SentencePieceProcessor()
sp.Load('../dataset/spm_10000_vocab_model.model')

True